In [1]:
import tensorflow as tf
from Audi_Classifier import Audio_Classifier

2023-12-27 23:42:56.848082: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 23:42:56.893966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 23:42:56.893990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 23:42:56.894961: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 23:42:56.901407: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 23:42:56.903084: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
data_path = "dataset"
train_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=60

)
validation_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=5
)


Found 1619 files belonging to 17 classes.
Using 1296 files for training.
Found 1619 files belonging to 17 classes.
Using 323 files for validation.


In [3]:
model = Audio_Classifier.create_model(train_data ,validation_data)
print(model)
Audio_Classifier.train(model,train_data ,validation_data)


None


AttributeError: 'NoneType' object has no attribute 'compile'

In [ ]:
model.save('audio_friends.h5')